In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.layers import Attention
from tensorflow.keras.optimizers import Adam


In [2]:
data = pd.read_csv("TamilSummarization.csv")  
data = data.dropna()

In [3]:
text_data = data['Text']
summary_data = data['Summary']

In [4]:
max_text_len = 100
max_summary_len = 15

In [13]:
summary_data = data['Summary'].apply(lambda x: 'start ' + x + ' end')
text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(text_data)
text_sequences = text_tokenizer.texts_to_sequences(text_data)
text_vocab_size = len(text_tokenizer.word_index) + 1

summary_tokenizer = Tokenizer()
summary_tokenizer.fit_on_texts(summary_data)
summary_sequences = summary_tokenizer.texts_to_sequences(summary_data)
summary_vocab_size = len(summary_tokenizer.word_index) + 1

In [14]:
# Pad sequences
text_sequences = pad_sequences(text_sequences, maxlen=max_text_len, padding='post')
summary_sequences = pad_sequences(summary_sequences, maxlen=max_summary_len, padding='post')

# Split the data
X_train, X_val, y_train, y_val = train_test_split(text_sequences, summary_sequences, test_size=0.2, random_state=42)

# Define the model
embedding_dim = 100
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_text_len,))
enc_emb = Embedding(text_vocab_size, embedding_dim, trainable=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# Attention Layer
attention = Attention()([encoder_outputs, encoder_outputs])

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(summary_vocab_size, embedding_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# Dense Layer
decoder_dense = Dense(summary_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Compile the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
# Train the model
batch_size = 64
epochs = 10  

history = model.fit(
    [X_train, y_train[:, :-1]],
    y_train[:, 1:],
    epochs=epochs,
    batch_size=batch_size,
    validation_data=([X_val, y_val[:, :-1]], y_val[:, 1:])
)

Epoch 1/10
2/2 [==============================] - 12s 2s/step - loss: 6.3263 - accuracy: 0.2589 - val_loss: 6.2847 - val_accuracy: 0.5595
Epoch 2/10
2/2 [==============================] - 1s 708ms/step - loss: 6.2663 - accuracy: 0.5560 - val_loss: 6.1784 - val_accuracy: 0.5595
Epoch 3/10
2/2 [==============================] - 1s 540ms/step - loss: 6.1054 - accuracy: 0.5560 - val_loss: 5.4548 - val_accuracy: 0.5595
Epoch 4/10
2/2 [==============================] - 1s 795ms/step - loss: 4.8815 - accuracy: 0.5560 - val_loss: 3.5275 - val_accuracy: 0.5595
Epoch 5/10
2/2 [==============================] - 1s 765ms/step - loss: 3.3259 - accuracy: 0.5560 - val_loss: 3.0714 - val_accuracy: 0.5595
Epoch 6/10
2/2 [==============================] - 1s 744ms/step - loss: 2.9948 - accuracy: 0.5560 - val_loss: 3.0256 - val_accuracy: 0.5595
Epoch 7/10
2/2 [==============================] - 1s 767ms/step - loss: 2.9077 - accuracy: 0.5560 - val_loss: 2.9737 - val_accuracy: 0.5595
Epoch 8/10
2/2 [======

In [16]:
model.save("tamil_text_summarization_model2.h5")

In [17]:
# Inference Model
encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len, latent_dim))

dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c]
)
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs, decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2, state_h2, state_c2]
)

In [21]:
def decode_sequence(input_seq):
    encoder_outs, state_h, state_c = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = summary_tokenizer.word_index['start']  # Correctly added 'start' token

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq, encoder_outs, state_h, state_c])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        # Get sampled word
        sampled_token = summary_tokenizer.index_word.get(sampled_token_index, '')  # Gracefully handle missing indices
        if sampled_token == 'end' or len(decoded_sentence.split()) >= max_summary_len:
            stop_condition = True
        else:
            decoded_sentence += " " + sampled_token

        # Update target sequence
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        state_h, state_c = h, c

    return decoded_sentence.strip()


In [22]:
# Test the model
test_text = "அரசர்கள் ஆளும் நாடுகளில் மக்கள் அமைதியாக வாழ்ந்தனர்."
test_seq = pad_sequences(text_tokenizer.texts_to_sequences([test_text]), maxlen=max_text_len, padding='post')
summary = decode_sequence(test_seq)
print("Generated Summary:", summary)

1/1 [==============================] - 0s 112ms/step
Generated Summary: 
